In [1]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv()
import os

print(os.getenv("OPENAI_API_KEY")[:10])

sk-proj-vI


# Objective
Demonstrate short‑term memory (via LangGraph checkpointer `InMemorySaver`) and long‑term memory (via `InMemoryStore`) along with **LangMem** tools for extracting/searching memories.


Make sure to have langmem installed in your environment.

```bash
uv pip install langmem==0.0.29
```

In [2]:
# Helper function to display a graph
def display_graph(graph):
    from IPython.display import Image, display

    try:
        display(Image(graph.get_graph().draw_mermaid_png()))
    except Exception:
        # This requires some extra dependencies and is optional
        pass

In [3]:

import os
import uuid
# LangGraph core
from langgraph.checkpoint.memory import InMemorySaver  # short-term memory checkpointer
from langgraph.store.memory import InMemoryStore       # long-term memory store
from langchain.chat_models import init_chat_model
from pprint import pprint
from langchain_core.tools import tool

# Prebuilt utilities (React-style tool-using agent)
from langgraph.prebuilt import create_react_agent

# LangMem tools
from langmem import create_manage_memory_tool, create_search_memory_tool

In [4]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

llm = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort='minimal')  # or your model
agent = create_react_agent(llm, tools=[])

#  What the InMemorySaver (Checkpointer) does

Think of InMemorySaver as a short-term memory manager or state persistence layer.

In [5]:
checkpointer = InMemorySaver()

config = {"configurable": {"thread_id": "t1"}}
agent = create_react_agent(llm, tools=[], checkpointer=checkpointer)
result = agent.invoke({"messages": ["hello, my name is Sahana"]}, config)
pprint(result)

{'messages': [HumanMessage(content='hello, my name is Sahana', additional_kwargs={}, response_metadata={}, id='820222de-dddf-41a4-bf19-937a5f9ded3d'),
              AIMessage(content='Hi Sahana — nice to meet you! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 13, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CV0o4Z9lUpi4kAnWGFAOlJzNhH1G6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--64f506a9-c43d-45e5-83d3-c6451b4a6a38-0', usage_metadata={'input_tokens': 13, 'output_tokens': 25, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning

In [6]:
result = agent.invoke({"messages": ["Do you know the meaning of my name?"]}, config)
pprint(result)

{'messages': [HumanMessage(content='hello, my name is Sahana', additional_kwargs={}, response_metadata={}, id='820222de-dddf-41a4-bf19-937a5f9ded3d'),
              AIMessage(content='Hi Sahana — nice to meet you! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 13, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CV0o4Z9lUpi4kAnWGFAOlJzNhH1G6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--64f506a9-c43d-45e5-83d3-c6451b4a6a38-0', usage_metadata={'input_tokens': 13, 'output_tokens': 25, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning

## Advanced Checkpointing Concepts

Now let's explore more advanced checkpointing features:
1. **Multiple Conversations**: Managing two different conversation threads simultaneously
2. **State History**: Viewing the complete history of checkpoints for a thread
3. **Time Travel/Replay**: Going back to a previous checkpoint and continuing from there


### 1. Multiple Conversations with Different Thread IDs

Let's create two separate conversation threads and show how they maintain independent state.


In [7]:
# Create a new checkpointer for this demo
checkpointer = InMemorySaver()
agent = create_react_agent(llm, tools=[], checkpointer=checkpointer)

# Thread 1: Conversation about Python
config_thread1 = {"configurable": {"thread_id": "conversation_1"}}
print("=== THREAD 1: First message ===")
result1 = agent.invoke(
    {"messages": ["Hi! I'm learning Python. Can you help me? Keep it short and concise"]}, 
    config_thread1
)
print(result1["messages"][-1].content)
print()

# Thread 2: Conversation about cooking
config_thread2 = {"configurable": {"thread_id": "conversation_2"}}
print("=== THREAD 2: First message ===")
result2 = agent.invoke(
    {"messages": ["Hello! I want to learn how to bake bread. Keep it short and concise"]}, 
    config_thread2
)
print(result2["messages"][-1].content)


=== THREAD 1: First message ===
Sure — what do you need help with? Give one specific question (concept, code error, short task) and I’ll answer concisely. Examples you can ask:
- Explain list comprehensions with one example.
- Fix this error: TypeError: 'int' object is not iterable (paste code).
- Write a function to reverse a string.
- Quick guide to classes in 5 lines.

Pick one.

=== THREAD 2: First message ===
Great — here’s a short, step-by-step basic loaf (no-knead or minimal-knead) you can try.

Ingredients (for one 9x5" loaf)
- 3 cups (360 g) all-purpose flour
- 1¼ tsp salt
- ¼ tsp instant yeast (or 1 tsp active dry yeast)
- 1⅔ cups (400 ml) lukewarm water

Quick method (no-knead)
1. Mix: In a large bowl, whisk flour, salt, and yeast. Add water and stir until a shaggy, sticky dough forms. Scrape sides and cover with plastic or a towel.
2. First rise: Let sit at room temp 12–18 hours (dough will double and have bubbles).
3. Shape: Lightly flour your hands and counter. Turn dough

In [8]:
# Continue Thread 1 - Python conversation
print("=== THREAD 1: Second message ===")
result1 = agent.invoke(
    {"messages": ["What are decorators?"]}, 
    config_thread1
)
print(result1["messages"][-1].content)

=== THREAD 1: Second message ===
Decorators are a way to modify or extend the behavior of functions or classes without changing their source code. They wrap a callable with another callable that runs code before and/or after (or replaces) the original.

Key points (concise):
- A decorator is a callable that takes a function (or class) and returns a new function (or class).
- Syntax: use @decorator above a function definition.
- Useful for logging, timing, access control, caching, etc.
- Preserve metadata with functools.wraps.

Minimal example:
```
from functools import wraps

def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        print("before")
        result = f(*args, **kwargs)
        print("after")
        return result
    return wrapper

@my_decorator
def greet(name):
    return f"Hello, {name}!"

print(greet("Ana"))  # prints before, after and returns "Hello, Ana!"
```

Decorator with arguments (factory):
```
def repeat(n):
    def deco(f):
        @wraps(

In [9]:
# Continue Thread 2 - Cooking conversation
print("=== THREAD 2: Second message ===")
result2 = agent.invoke(
    {"messages": ["What ingredients do I need?"]}, 
    config_thread2
)
print(result2["messages"][-1].content)

=== THREAD 2: Second message ===
For a basic no-knead 9x5" (or similar) loaf:

- 3 cups (360 g) all-purpose flour  
- 1¼ teaspoons salt  
- ¼ teaspoon instant yeast (or 1 teaspoon active dry yeast)  
- 1⅔ cups (400 ml) lukewarm water

Optional add-ins (choose any):  
- 1–2 tablespoons sugar or honey (slightly sweeter)  
- 1–3 tablespoons olive oil or melted butter (richer crumb)  
- Up to 25% whole-wheat flour (replace part of the all-purpose)  
- Herbs, seeds, nuts, olives, or cheese (to taste)

If you want a different loaf type (sandwich, sourdough starter, or quick yeast loaf) I can list those ingredient sets.


In [10]:
# Resume Thread 1 again - it should remember the Python context
print("=== THREAD 1: Third message (resuming Python conversation) ===")
result1 = agent.invoke(
    {"messages": ["Can you give me a simple example?"]}, 
    config_thread1
)
print(result1["messages"][-1].content)
print()

print("✅ Notice how Thread 1 maintains context about Python and decorators!")
print("   Each thread maintains its own independent conversation history.")


=== THREAD 1: Third message (resuming Python conversation) ===
Sure — very simple logging decorator:

```
from functools import wraps

def log_calls(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with {args} {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result!r}")
        return result
    return wrapper

@log_calls
def add(a, b):
    return a + b

print(add(2, 3))
```

What happens:
- When you call add(2, 3), wrapper runs instead.
- It prints the call, runs the original add, prints the result, and returns it.

Want an example with decorator arguments or class methods?

✅ Notice how Thread 1 maintains context about Python and decorators!
   Each thread maintains its own independent conversation history.


### 2. Getting State History

Let's examine the complete checkpoint history for Thread 1 to see every state saved during the conversation.


In [11]:
# Get the state history for Thread 1
state_history = list(agent.get_state_history(config_thread1))

print(f"📊 Total checkpoints in Thread 1: {len(state_history)}")
print("=" * 80)

def print_checkpoint_details(state_history, idx):
    """
    Print details of the checkpoint at the given index from state_history.
    """
    if idx < 0 or idx >= len(state_history):
        print(f"❌ Invalid index {idx}. There are {len(state_history)} checkpoints.")
        return
    
    state = state_history[idx]
    print(f"\n🔖 Checkpoint {idx + 1}:")
    print(f"   Checkpoint ID: {state.config['configurable']['checkpoint_id'][:20]}...")
    print(f"   Messages in state: {len(state.values.get('messages', []))}")
    
    # Show the last message in this checkpoint
    if state.values.get('messages'):
        last_msg = state.values['messages'][-1]
        content_preview = last_msg.content[:100] + "..." if len(last_msg.content) > 100 else last_msg.content
        print(f"   Last message: [{last_msg.__class__.__name__}] {content_preview}")
    
    print(f"   Next nodes: {state.next}")
    print(f"   Metadata: {state.metadata}")

# Example usage:
print_checkpoint_details(state_history, 0)

📊 Total checkpoints in Thread 1: 9

🔖 Checkpoint 1:
   Checkpoint ID: 1f0b2a2e-f462-6f6c-8...
   Messages in state: 6
   Last message: [AIMessage] Sure — very simple logging decorator:

```
from functools import wraps

def log_calls(func):
    @wr...
   Next nodes: ()
   Metadata: {'source': 'loop', 'writes': {'agent': {'messages': [AIMessage(content='Sure — very simple logging decorator:\n\n```\nfrom functools import wraps\n\ndef log_calls(func):\n    @wraps(func)\n    def wrapper(*args, **kwargs):\n        print(f"Calling {func.__name__} with {args} {kwargs}")\n        result = func(*args, **kwargs)\n        print(f"{func.__name__} returned {result!r}")\n        return result\n    return wrapper\n\n@log_calls\ndef add(a, b):\n    return a + b\n\nprint(add(2, 3))\n```\n\nWhat happens:\n- When you call add(2, 3), wrapper runs instead.\n- It prints the call, runs the original add, prints the result, and returns it.\n\nWant an example with decorator arguments or class methods?', addition

### Limitations of InMemorySaver**: 
- Data is lost when the process ends
- Not suitable for production (use `SqliteSaver` or `PostgresSaver` instead)
- Good for development, testing, and notebooks

For more details, see the [LangGraph Persistence Documentation](https://langchain-ai.github.io/langgraph/concepts/persistence/#threads).

#  What can InMemoryStore do? 



The In-Memory Store persists memory within a session, but its real value lies in how it organizes and structures information. Unlike the In-Memory Saver, which mainly captures conversation flow for debugging or replay, the In-Memory Store lets an agent store facts in a structured way and retrieve them later as part of decision-making.

In [12]:
from langchain.embeddings import init_embeddings
embeddings = init_embeddings("openai:text-embedding-3-small")

store = InMemoryStore(index={
        "embed": embeddings,
        "dims": 1536,
    })


@tool
def save_order(item: str, quantity: int = 1, table_no: str = "unknown"):
    """
    Saves a restaurant order into the InMemoryStore.
    """
    namespace = ("restaurant", "orders")
    order_id = str(uuid.uuid4())
    store.put(namespace, order_id, {
        "item": item,
        "quantity": quantity,
        "table": table_no
    })
    return f"✅ Saved order for {quantity} x {item} (Table {table_no})."

In [13]:
agent = create_react_agent(
    llm,
    tools=[save_order],       
    store=store
)

In [14]:

def ask(text):
    out = agent.invoke({"messages": [{"role": "user", "content": text}]})
    print(out["messages"][-1].content)

# Simulate a few interactions
ask("Save an order for 1 butterscotch ice cream at table 5.")
ask("Save an order for 3 oat milk lattes at table 2.")
ask("Save an orer for 3 avocado toast at table 2.")

Done — I saved the order: 1 butterscotch ice cream for table 5.
Done — I saved the order: 3 oat milk lattes for table 2. Anything else to add or change?
Done — saved order for 3 avocado toast at table 2.


In [15]:
items = store.search(
     ("restaurant", "orders"), query="what is table 5 order", limit=1)
pprint(items)

[Item(namespace=['restaurant', 'orders'], key='379581f4-86a0-4872-941d-7ef25e7a7ca9', value={'item': 'butterscotch ice cream', 'quantity': 1, 'table': '5'}, created_at='2025-10-26T19:35:34.674782+00:00', updated_at='2025-10-26T19:35:34.674788+00:00', score=0.41818210641875714)]


## InMemoryStore = long-term structure, not long-term persistence. 

In the In-Memory Store setup, we manually created a tool (like save_order) to save items into the store.
For retrieval, we also had to manage the namespace, craft the search query, and sometimes tweak parameters such as the number of results or filtering logic.

LangMem takes that abstraction away.
It lets the agent itself decide when to remember something and how to retrieve it — without us manually calling store.put() or store.search().

In [18]:
from langchain.embeddings import init_embeddings
from langmem import create_manage_memory_tool, create_search_memory_tool

embeddings = init_embeddings("openai:text-embedding-3-small")

new_store = InMemoryStore(index={
        "embed": embeddings,
        "dims": 1536,
    })

agent = create_react_agent(
    llm,
    tools=[create_manage_memory_tool(namespace=("memories",)),
        create_search_memory_tool(namespace=("memories",)),],       
    store=new_store
)


In [19]:

def ask(text):
    out = agent.invoke({"messages": [{"role": "user", "content": text}]})
    print(out["messages"][-1].content)

# Simulate a few interactions
ask("Save an order for 2 cappuccinos at table 3.")
ask("Save an order for 1 butterscotch ice cream at table 5.")
ask("Save an order for 3 oat milk lattes at table 2.")
ask("Save an orer for 3 avocado toast at table 2.")

Done — I saved the order: 2 cappuccinos for table 3 (pending). Would you like to add modifications, set a priority, or send it to the kitchen?
Done — I saved the order for 1 butterscotch ice cream at table 5.
Got it — I saved the order: 3 oat milk lattes for table 2. Would you like me to add anything else (modifiers, time to serve, or customer name)?
Done — I saved the order: 3 avocado toast for table 2. Would you like to add any modifiers (toasts doneness, extras, allergies) or set a pickup/serve time?


In [23]:
result = agent.invoke({"messages": [{"role": "user", "content": "What did table 5 order earlier, look up your saved memories"}]})
pprint(result["messages"][-1].content)


'Table 5 ordered 1 butterscotch ice cream.'


In [24]:
store.search(("restaurant", "orders"), query="table 5 orders", limit=1)

[Item(namespace=['restaurant', 'orders'], key='379581f4-86a0-4872-941d-7ef25e7a7ca9', value={'item': 'butterscotch ice cream', 'quantity': 1, 'table': '5'}, created_at='2025-10-26T19:35:34.674782+00:00', updated_at='2025-10-26T19:35:34.674788+00:00', score=0.4361556401381326)]

### More Control via MemoryManager in LangMem

In [27]:
from langmem import create_memory_manager

manager = create_memory_manager(
    "openai:gpt-4o",
    instructions="Extract all noteworthy facts, events, and relationships. Indicate their importance.",
    enable_updates=True,
)

# Process a conversation to extract semantic memories
conversation = [
    {"role": "user", "content": "I work at Acme Corp in the ML team"},
    {"role": "assistant", "content": "I'll remember that. What kind of ML work do you do?"},
    {"role": "user", "content": "Mostly NLP and large language models"}
]

memories = manager.invoke({"messages": conversation})
for memory in memories:
    print(memory)

ExtractedMemory(id='2670fd63-a81a-4643-834d-fdce4653115e', content=Memory(content='User works at Acme Corp in the ML team, primarily focusing on NLP and large language models.'))


In [26]:
store = InMemoryStore(index={
        "embed": embeddings,
        "dims": 1536,
    })

conversation = [
    {"role": "user", "content": "I like travelling to Europe"},
    {"role": "assistant", "content": "Amazing, I can suggest some places to visit"},
]

new_memories = manager.invoke({"messages": conversation, "existing":memories})
for memory in new_memories:
    print(memory)

ExtractedMemory(id='bd939918-d259-4c82-bf54-8b98f3311af2', content=Memory(content='The user likes travelling to Europe.'))
ExtractedMemory(id='adce12c0-1306-49b0-ab6f-93f009c1adc6', content=Memory(content='The user works at Acme Corp in the Machine Learning (ML) team.'))
ExtractedMemory(id='5620d281-06c1-401a-8cd6-bca935ebb524', content=Memory(content="The user's work in the Machine Learning (ML) team at Acme Corp primarily involves Natural Language Processing (NLP) and large language models."))


In [28]:
conversation = [
    {"role": "user", "content": "I switched jobs, I'm working with LevelupLabs now"},
    {"role": "assistant", "content": "That's great!"},
]

new_memories = manager.invoke({"messages": conversation, "existing":memories})
for memory in new_memories:
    print(memory)

ExtractedMemory(id='7105d6f3-66dc-48d1-a84a-d8e4477c064a', content=Memory(content='The user has switched jobs and is now working at LevelupLabs.'))
ExtractedMemory(id='2670fd63-a81a-4643-834d-fdce4653115e', content=Memory(content='User works at LevelupLabs in the ML team, primarily focusing on NLP and large language models.'))


In [29]:

# Structured memory extraction example: Extracting episodic memory with custom schema

from pydantic import BaseModel, Field
from langmem import create_memory_manager

# Define a custom Pydantic schema for episodic memory
class EpisodicMemory(BaseModel):
    summary: str = Field(description="A summary of the conversation")
    key_insights: list[str] = Field(description="Key insights from the conversation")
    user_preferences: list[str] = Field(description="User preferences mentioned in the conversation, like travel destinations, food preferences, etc.")
    search_keywords: list[str] = Field(description="keywords used for the searching the memory from the conversation")

# Create a memory manager with the custom schema
episodic_manager = create_memory_manager(
    "openai:gpt-4o",
    schemas=[EpisodicMemory],
    instructions="Extract very specific episodic memories in the requested format.",
    enable_inserts=True,
)

episodic_conversation = [
    {"role": "user", "content": "Yesterday at the coffee shop, I met Alice and we discussed our weekend plans."},
    {"role": "assistant", "content": "Sounds like you and Alice had a productive talk!"},
    {"role": "user", "content": "Yes, we decided to go hiking on Saturday."},
]

episodic_memories = episodic_manager.invoke({"messages": episodic_conversation})
for m in episodic_memories:
    print(m)

ExtractedMemory(id='51554b04-e45a-43b4-b355-9f03b1218ea2', content=EpisodicMemory(summary='User met Alice at a coffee shop to discuss weekend plans.', key_insights=['User plans to go hiking with Alice on Saturday.'], user_preferences=['hiking'], search_keywords=['Alice', 'coffee shop', 'hiking', 'weekend plans']))


In [30]:
store.put(("episodic", "memories"), episodic_memories[0].id, episodic_memories[0].content.model_dump_json())

In [31]:
store.search(("episodic", "memories"), query="Alice", limit=3)

[Item(namespace=['episodic', 'memories'], key='51554b04-e45a-43b4-b355-9f03b1218ea2', value='{"summary":"User met Alice at a coffee shop to discuss weekend plans.","key_insights":["User plans to go hiking with Alice on Saturday."],"user_preferences":["hiking"],"search_keywords":["Alice","coffee shop","hiking","weekend plans"]}', created_at='2025-10-26T19:46:39.858957+00:00', updated_at='2025-10-26T19:46:39.858966+00:00', score=0.27427522006147353)]